note: ensure both endpoints are running

In [2]:
import boto3
import sagemaker
from sagemaker import Session
from sagemaker.session import TrainingInput
from sagemaker.tensorflow import TensorFlow as tf
from sagemaker.tensorflow import TensorFlowModel, TensorFlowPredictor
from sagemaker.tuner import HyperparameterTuner, IntegerParameter, ContinuousParameter, CategoricalParameter
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer
import tensorflow
import s3fs
import numpy as np
import json
import time
RANDOM_SEED = 0
sagemaker.__version__
fs = s3fs.S3FileSystem()
sm = boto3.client("sagemaker")
sess = sagemaker.Session()
role = sagemaker.get_execution_role()
s3 = boto3.client("s3")
bucket = sess.default_bucket()   # will be sagemaker-ap-southeast-2-838084669510
prefix = "aiornot"
print(f"S3 Bucket: f{bucket}")


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


2025-08-16 12:23:37.729102: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


S3 Bucket: fsagemaker-ap-southeast-2-838084669510


# Start Endpoints

In [3]:
transfer_model_s3_path = "s3://sagemaker-ap-southeast-2-838084669510/model_output/transfer-learning-20250816-075841/output/model.tar.gz"
transfer_model = TensorFlowModel(
    model_data=transfer_model_s3_path,
    role=role,
    framework_version="2.14"
)

try:
    transfer_model_predictor = transfer_model.deploy(
        initial_instance_count=1,
        instance_type="ml.m5.large",
        endpoint_name="transfer-model-endpoint"
    )
except Exception: # has already been deployed, so just call the existing endpoint
    transfer_model_predictor = TensorFlowPredictor(
        endpoint_name="transfer-model-endpoint",
        sagemaker_session=sess
    )

In [4]:
main_model_s3_path = "s3://sagemaker-ap-southeast-2-838084669510/aiornot/model_output/bestparams-refit-20250816-094717/output/model.tar.gz"
main_model = TensorFlowModel(
    model_data=main_model_s3_path,
    role=role,
    framework_version="2.14"
)
try: # has not yet been deployed
    main_model_predictor = main_model.deploy(
        initial_instance_count=1,
        instance_type="ml.m5.large",
        endpoint_name="final-model-endpoint"
    )
except Exception: # has already been deployed, so just call the existing endpoint
    main_model_predictor = TensorFlowPredictor(
        endpoint_name="final-model-endpoint",
        sagemaker_session=sess
    )

# Make Predictions for both models

In [ ]:
test_path = f"s3://{bucket}/{prefix}/small_train/small_train.npz"

with fs.open(test_path, "rb") as f:
    d = np.load(f)
    X = d["image"].astype("float32") 
    y_true = np.asarray(d["label"], dtype=int).ravel()
    print("data loaded")

def predict_batches(pred, X, bs=1):
    probs = []
    for i in range(0, len(X), bs):
        out = pred.predict(X[i:i+bs].tolist())
        p = np.array(out.get("predictions", out)).reshape(-1)  # shape (bs,)
        probs.append(p)
        print(f"{i}/{len(X)}")
    probs = np.concatenate(probs)
    predictions = (probs >= 0.5).astype(int)
    return predictions


y_test_main_model = predict_batches(main_model_predictor, X)
y_test_transfer_model = predict_batches(transfer_model_predictor, X)